In [96]:
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm

import astropy.constants as c 
import astropy.units as u

%run PS4Q2Funcs.ipynb

In [97]:
KB = c.k_B.cgs.value
PI = np.pi
E_B = (4.52 * u.eV).cgs.value
H = c.h.cgs.value
HBAR = H/(2*PI)
W_VIB = 5.4e14
MOI = 9.1e-41
I = (13.6 * u.eV).cgs.value
A0 = c.a0.cgs.value
M_H = 1.67e-24
M_P = c.m_p.cgs.value
M_E = c.m_e.cgs.value

In [98]:
Temp = np.logspace(3, 4, 100)
Pressure = np.logspace(3.5, 9, 100)

xh1 = np.full((len(Pressure), len(Temp)), np.nan)
xh11 = np.full((len(Pressure), len(Temp)), np.nan)
h2entropy = np.full((len(Pressure), len(Temp)), np.nan)
hIentropy = np.full((len(Pressure), len(Temp)), np.nan)
hIIentropy = np.full((len(Pressure), len(Temp)), np.nan)

In [99]:
for indexP, P in enumerate(Pressure):
    for indexT, T in enumerate(Temp):
        xh1[indexP, indexT] = ion_frac(T, P)

xh1


array([[6.43938670e-08, 1.16868516e-07, 2.09285391e-07, ...,
        9.99999419e-01, 9.99999493e-01, 9.99999557e-01],
       [6.04041434e-08, 1.09627561e-07, 1.96318460e-07, ...,
        9.99999339e-01, 9.99999424e-01, 9.99999497e-01],
       [5.66616155e-08, 1.02835243e-07, 1.84154936e-07, ...,
        9.99999249e-01, 9.99999345e-01, 9.99999428e-01],
       ...,
       [1.30136786e-10, 2.36185434e-10, 4.22955342e-10, ...,
        8.87713093e-01, 8.99485643e-01, 9.10032181e-01],
       [1.22073753e-10, 2.21551826e-10, 3.96749819e-10, ...,
        8.75793827e-01, 8.88533822e-01, 8.99997312e-01],
       [1.14510292e-10, 2.07824888e-10, 3.72167941e-10, ...,
        8.62953064e-01, 8.76680129e-01, 8.89088866e-01]], shape=(100, 100))

In [100]:
for indexP, P in enumerate(Pressure):
    for indexT, T in enumerate(Temp):
        xh11[indexP, indexT] = ion_h11(T,P)

xh11

array([[3.10092336e-33, 1.95883277e-32, 1.18683486e-31, ...,
        2.20011113e-01, 2.65734398e-01, 3.17519709e-01],
       [2.90879593e-33, 1.83746715e-32, 1.11330079e-31, ...,
        2.07966934e-01, 2.51649430e-01, 3.01353704e-01],
       [2.72857237e-33, 1.72362112e-32, 1.04432275e-31, ...,
        1.96494019e-01, 2.38180485e-01, 2.85821266e-01],
       ...,
       [6.26681087e-36, 3.95870297e-35, 2.39853384e-34, ...,
        5.03432557e-04, 6.26727605e-04, 7.76805822e-04],
       [5.87853096e-36, 3.71342912e-35, 2.24992516e-34, ...,
        4.72248183e-04, 5.87908150e-04, 7.28693926e-04],
       [5.51430816e-36, 3.48335199e-35, 2.11052400e-34, ...,
        4.42995049e-04, 5.51492506e-04, 6.83560847e-04]], shape=(100, 100))

In [101]:
for indexP, P in enumerate(Pressure):
    for indexT, T in enumerate(Temp):
        f_h2 = f_h2_noconst(P, xh1[indexP, indexT], xh11[indexP, indexT])
        entr_h2 = float(derivative(f_h2, T))
        h2entropy[indexP, indexT] = KB / (2 * M_H) * float(entr_h2)
        
h2entropy

/var/folders/bh/ph3t25_s225g7tsmf91j29w80000gn/T/ipykernel_18471/818651056.py:4: DeprecationWarning: scipy.misc.derivative is deprecated in SciPy v1.10.0; and will be completely removed in SciPy v1.12.0. You may consider using findiff: https://github.com/maroba/findiff or numdifftools: https://github.com/pbrod/numdifftools
  entr_h2 = float(derivative(f_h2, T))
/var/folders/bh/ph3t25_s225g7tsmf91j29w80000gn/T/ipykernel_18471/3991760740.py:8: RuntimeWarning: invalid value encountered in log
  return T * np.log(ln)


array([[8.22997053e+08, 8.25681123e+08, 8.28379749e+08, ...,
                   nan,            nan,            nan],
       [8.17709192e+08, 8.20393262e+08, 8.23091888e+08, ...,
                   nan,            nan,            nan],
       [8.12421331e+08, 8.15105401e+08, 8.17804027e+08, ...,
                   nan,            nan,            nan],
       ...,
       [3.10074570e+08, 3.12758638e+08, 3.15457260e+08, ...,
        7.07763585e+08, 7.15888305e+08, 7.24054458e+08],
       [3.04786709e+08, 3.07470777e+08, 3.10169400e+08, ...,
        6.98277156e+08, 7.06285416e+08, 7.14339287e+08],
       [2.99498848e+08, 3.02182917e+08, 3.04881539e+08, ...,
        6.88898925e+08, 6.96786719e+08, 7.04724976e+08]], shape=(100, 100))

In [102]:
for indexP, P in enumerate(Pressure):
    for indexT, T in enumerate(Temp):
        f_hI = f_hI_noconst(P, xh1[indexP, indexT])
        entr_hI = float(derivative(f_hI, T))
        hIentropy[indexP, indexT] = KB / (2 * M_H) * float(entr_hI)
        
hIentropy

/var/folders/bh/ph3t25_s225g7tsmf91j29w80000gn/T/ipykernel_18471/4145669515.py:4: DeprecationWarning: scipy.misc.derivative is deprecated in SciPy v1.10.0; and will be completely removed in SciPy v1.12.0. You may consider using findiff: https://github.com/maroba/findiff or numdifftools: https://github.com/pbrod/numdifftools
  entr_hI = float(derivative(f_hI, T))


array([[1.31944788e+09, 1.29625200e+09, 1.27360929e+09, ...,
        7.75197159e+08, 7.76777671e+08, 7.78411299e+08],
       [1.31680394e+09, 1.29360807e+09, 1.27096536e+09, ...,
        7.69909302e+08, 7.71489813e+08, 7.73123441e+08],
       [1.31416001e+09, 1.29096414e+09, 1.26832143e+09, ...,
        7.64588381e+08, 7.66154695e+08, 7.67768625e+08],
       ...,
       [1.06298663e+09, 1.03979076e+09, 1.01714805e+09, ...,
        2.66883483e+08, 2.67784686e+08, 2.68749651e+08],
       [1.06034270e+09, 1.03714683e+09, 1.01450412e+09, ...,
        2.62154408e+08, 2.63003217e+08, 2.63920141e+08],
       [1.05769877e+09, 1.03450290e+09, 1.01186019e+09, ...,
        2.57477109e+08, 2.58270531e+08, 2.59136365e+08]], shape=(100, 100))

In [103]:
for indexP, P in enumerate(Pressure):
    for indexT, T in enumerate(Temp):
        f_hII = f_hII_noconst(P, xh11[indexP, indexT])
        entr_hII = float(derivative(f_hII, T))
        hIIentropy[indexP, indexT] = KB / (2 * M_P) * float(entr_hII)
        
hIIentropy

/var/folders/bh/ph3t25_s225g7tsmf91j29w80000gn/T/ipykernel_18471/1806739912.py:4: DeprecationWarning: scipy.misc.derivative is deprecated in SciPy v1.10.0; and will be completely removed in SciPy v1.12.0. You may consider using findiff: https://github.com/maroba/findiff or numdifftools: https://github.com/pbrod/numdifftools
  entr_hII = float(derivative(f_hII, T))


array([[3.69483518e+09, 3.62020111e+09, 3.54728840e+09, ...,
        8.07634824e+08, 8.01281759e+08, 7.95373469e+08],
       [3.69219539e+09, 3.61756133e+09, 3.54464862e+09, ...,
        8.04678823e+08, 7.98249875e+08, 7.92250575e+08],
       [3.68955561e+09, 3.61492154e+09, 3.54200883e+09, ...,
        8.01741318e+08, 7.95240603e+08, 7.89155035e+08],
       ...,
       [3.43877595e+09, 3.36414189e+09, 3.29122918e+09, ...,
        5.46449452e+08, 5.38848198e+08, 5.31427874e+08],
       [3.43613617e+09, 3.36150210e+09, 3.28858939e+09, ...,
        5.43809022e+08, 5.36207611e+08, 5.28787095e+08],
       [3.43349638e+09, 3.35886232e+09, 3.28594961e+09, ...,
        5.41168632e+08, 5.33567073e+08, 5.26146377e+08]], shape=(100, 100))